# Lesson 2: Guaranteed Structure with the Gemini API

Welcome to this lesson on the most robust method for getting structured, predictable output from the Gemini API.

### The Problem
When you ask an LLM for data, its default text response can be inconsistent and filled with conversational text, making it difficult to use in an application.

### The Solution: Response Schemas
Instead of just asking for JSON in the prompt, we can provide a **`response_schema`**. This is a programmatic definition (like a Python class) of the exact data structure we want. This forces the model to return a predictable Python object, which is even better than a JSON string.

In this notebook, we'll first see the problem with unstructured text and then solve it using this powerful schema-based approach.

In [16]:
#@title 1. Setup
# Install the Google AI Python SDK
!pip install -q -U google-genai

# Import necessary libraries
from google import genai
import json
import typing
from google.colab import userdata

In [7]:
#@title 2. Configure your API Key
# To use the Gemini API, you need an API key.
# You can get one from Google AI Studio: https://aistudio.google.com/app/apikey
#
# Use the "Secrets" tab in Colab (click the key icon on the left) to store your
# API key with the name "GOOGLE_API_KEY". This is more secure than pasting it directly.

try:
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
    print('Secret not found. Please add your GOOGLE_API_KEY to the Colab Secrets Manager.')
    # Or, uncomment and paste your key here for a quick test:
    # GOOGLE_API_KEY="YOUR_API_KEY"

## Part 1: The Problem - Unpredictable Text Output

First, let's see what happens when we ask a standard model for product information. The output is conversational and has no consistent structure.

In [13]:
# Our unstructured text data
unstructured_text = """
The new Aero-Drone X is an amazing piece of technology. It features a high-resolution 4K camera,
an impressive 30-minute flight time, and advanced obstacle avoidance sensors.
It is available for purchase now at a price of $499.99. Don't miss out!
"""


# A simple prompt asking for information
prompt_for_text = f"Extract the key information from this description: {unstructured_text}"

print(f"Sending a simple prompt:\n'{prompt_for_text}'\n")

# Get the unpredictable, conversational response

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt_for_text
)

Sending a simple prompt:
'Extract the key information from this description: 
The new Aero-Drone X is an amazing piece of technology. It features a high-resolution 4K camera, 
an impressive 30-minute flight time, and advanced obstacle avoidance sensors. 
It is available for purchase now at a price of $499.99. Don't miss out!
'



In [14]:
#@title See the messy, unstructured output
print(response.text)

print("\n\n---")
print("Notice how this is just a wall of text? To get the price or features,")
print("we would have to write complex code to parse this string.")

Here's the key information from the description:

*   **Product Name:** Aero-Drone X
*   **Key Features:**
    *   High-resolution 4K camera
    *   30-minute flight time
    *   Advanced obstacle avoidance sensors
*   **Price:** $499.99
*   **Availability:** Available for purchase now


---
Notice how this is just a wall of text? To get the price or features,
we would have to write complex code to parse this string.


## Part 2: The Solution - Using a Response Schema

Now, let's solve this problem. We'll define our desired data structure as a Python class and pass it to the model. This forces the model to structure its response according to our exact specifications.

In [26]:
#@title 1. Define the Response Schema
# We define our desired structure using standard Python typing.
# This class tells the model exactly what fields to include and their types.
from pydantic import BaseModel

class Product(BaseModel):
  product_name: str
  features: list[str]
  price: float
  availability: str

print("Schema defined successfully!")

Schema defined successfully!


In [27]:
#@title 2. Configure and Initialize the Model
# We will use a model that is optimized for following structured output instructions.
# The key step is passing our Product class as the response_schema.

# The prompt can now be much simpler. We don't need to describe the JSON format.
prompt_for_schema = f"Extract the product information from this text: {unstructured_text}"

# The response_schema should be a dictionary that defines the schema.
# The SDK will convert the NamedTuple to the correct dictionary format.
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt_for_schema,
    config={
        "response_mime_type": "application/json",
        "response_schema": Product
    },
)

In [29]:
#@title 3. See the clean, structured output
# The model's response is no longer just text. It's a structured Python object
# that perfectly matches the schema we defined.
product_object = response.parsed

print("--- The model returned a structured object ---")
print(product_object)

print("\n\n--- We can now easily access the data ---")
print(f"Product Name: {product_object.product_name}\n")

print("Features:")
for feature in product_object.features:
  print(f"- {feature}")

print(f"\nPrice: ${product_object.price}")

--- The model returned a structured object ---
product_name='Aero-Drone X' features=['high-resolution 4K camera', '30-minute flight time', 'advanced obstacle avoidance sensors'] price=499.99 availability='available for purchase now'


--- We can now easily access the data ---
Product Name: Aero-Drone X

Features:
- high-resolution 4K camera
- 30-minute flight time
- advanced obstacle avoidance sensors

Price: $499.99
